In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# Пути к файлам
scores_file = r"../../../depmap_datasets/prepared_datasets/depscores.csv"
expression_file = r"../../../depmap_datasets/prepared_datasets/protein_coding_gene_expression.csv"
correlation_output_file = r"./correlation_depscore_vs_expression_data.csv"
pvalue_output_file = r"./p_value_depscore_vs_expression_data.csv"

# Список генов для анализа
genes_of_interest = [
    "ABL1", "ABL2", "AXL", "BMX", "BTK", "CAMK1D", "CAMK2G", "CAV1", "CSK", 
    "EGFR", "EPHB4", "FER", "FES", "FGR", "FRK", "FYN", "HCK", "LCK", "LYN", "MAP4K2", 
    "MAP4K5", "MINK1", "PTK2", "PTK2B", "SIK1", "SIK2", "SIK3", "SRC", "STK10", 
    "STK24", "STK4", "SYK", "TBK1", "TEC", "TNK2", "YES1"
]

# Загрузка данных
scores_data = pd.read_csv(scores_file)
expression_data = pd.read_csv(expression_file)
scores_data = scores_data.rename(columns={"DepMap_ID": "ModelID"})

# Отбор только интересующих генов
scores_data_filtered = scores_data[["ModelID"] + [gene for gene in genes_of_interest if gene in scores_data.columns]]
expression_data_filtered = expression_data[["ModelID"] + [gene for gene in genes_of_interest if gene in expression_data.columns]]

# Объединение по ModelID
merged_data = pd.merge(scores_data_filtered, expression_data_filtered, on="ModelID", suffixes=('_depscore', '_expression'))

# Разделение данных на оценки зависимости и экспрессию
scores_only = merged_data.filter(like='_depscore').copy()
expression_only = merged_data.filter(like='_expression').copy()

# Подготовка таблиц для корреляций и p-value
correlation_matrix = pd.DataFrame(index=scores_only.columns, columns=expression_only.columns, dtype=float)
pvalue_matrix = pd.DataFrame(index=scores_only.columns, columns=expression_only.columns, dtype=float)

# Расчёт корреляций и p-value
for score_gene in scores_only.columns:
    for expr_gene in expression_only.columns:
        x = scores_only[score_gene]
        y = expression_only[expr_gene]
        mask = x.notna() & y.notna()
        if mask.sum() >= 10:
            corr, pval = pearsonr(x[mask], y[mask])
            correlation_matrix.loc[score_gene, expr_gene] = corr
            pvalue_matrix.loc[score_gene, expr_gene] = pval
        else:
            correlation_matrix.loc[score_gene, expr_gene] = np.nan
            pvalue_matrix.loc[score_gene, expr_gene] = np.nan

# Сохранение результатов
correlation_matrix.to_csv(correlation_output_file)
pvalue_matrix.to_csv(pvalue_output_file)

print(f"✅ Корреляционная матрица сохранена: {correlation_output_file}")
print(f"✅ P-value матрица сохранена: {pvalue_output_file}")


✅ Корреляционная матрица сохранена: ./correlation_depscore_vs_expression_data.csv
✅ P-value матрица сохранена: ./p_value_depscore_vs_expression_data.csv
